In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [2]:
# Cargar las películas y las calificaciones
movies = pd.read_csv('/content/movie.csv')
ratings = pd.read_csv('/content/rating.csv')

# Explorar las primeras filas de cada dataset
print(movies.head())
print(ratings.head())


   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [3]:
# Extraer las películas que son de terror
horror_movies = movies[movies['genres'].str.contains('Horror')]

# Revisar las películas de terror
print(horror_movies.head())


    movieId                               title  \
11       12  Dracula: Dead and Loving It (1995)   
21       22                      Copycat (1995)   
69       70          From Dusk Till Dawn (1996)   
90       92                  Mary Reilly (1996)   
91       93          Vampire in Brooklyn (1995)   

                                 genres  
11                        Comedy|Horror  
21  Crime|Drama|Horror|Mystery|Thriller  
69        Action|Comedy|Horror|Thriller  
90                Drama|Horror|Thriller  
91                Comedy|Horror|Romance  


In [4]:
#¿Qué película recomendarían si se quiere ver una película de terror?
# Recomendar las películas de terror más populares basadas en el número de calificaciones
horror_ratings = ratings[ratings['movieId'].isin(horror_movies['movieId'])]
horror_ratings_grouped = horror_ratings.groupby('movieId').size().reset_index(name='count')
top_horror_movies = horror_ratings_grouped.sort_values(by='count', ascending=False).head(10)

# Unir con la información de películas
top_horror_movies = pd.merge(top_horror_movies, movies, on='movieId')
print(top_horror_movies[['title', 'genres', 'count']])


                                               title  \
0                   Silence of the Lambs, The (1991)   
1                            Sixth Sense, The (1999)   
2                                       Alien (1979)   
3                                      Aliens (1986)   
4  Interview with the Vampire: The Vampire Chroni...   
5                                Shining, The (1980)   
6                                        Jaws (1975)   
7                    Blair Witch Project, The (1999)   
8                                      Psycho (1960)   
9                                      Scream (1996)   

                           genres  count  
0           Crime|Horror|Thriller   1075  
1            Drama|Horror|Mystery    665  
2                   Horror|Sci-Fi    526  
3  Action|Adventure|Horror|Sci-Fi    503  
4                    Drama|Horror    497  
5                          Horror    410  
6                   Action|Horror    390  
7           Drama|Horror|Thriller    35

In [5]:
#¿Qué película recomendarían si mi última película fue Toy Story?
# Crear una tabla de usuario-película
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Buscar el ID de la película 'Toy Story'
toy_story_id = movies[movies['title'].str.contains('Toy Story')]['movieId'].values[0]

# Calcular la similitud entre películas utilizando la similitud del coseno
movie_similarity = cosine_similarity(user_movie_matrix.fillna(0).T)

# Crear un DataFrame para ver las películas más similares
similarity_df = pd.DataFrame(movie_similarity, index=user_movie_matrix.columns, columns=user_movie_matrix.columns)

# Obtener las películas más similares a Toy Story
similar_movies = similarity_df[toy_story_id].sort_values(ascending=False).head(10)

# Unir con la información de películas para ver los títulos
similar_movies = pd.merge(similar_movies.reset_index(), movies, left_on='movieId', right_on='movieId')
print(similar_movies[['title', 'genres']])


                                               title  \
0                                   Toy Story (1995)   
1          Star Wars: Episode IV - A New Hope (1977)   
2               Independence Day (a.k.a. ID4) (1996)   
3  Star Wars: Episode VI - Return of the Jedi (1983)   
4                          Back to the Future (1985)   
5                                 Toy Story 2 (1999)   
6                               Jurassic Park (1993)   
7                                Forrest Gump (1994)   
8                  Terminator 2: Judgment Day (1991)   
9  Star Wars: Episode V - The Empire Strikes Back...   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                      Action|Adventure|Sci-Fi  
2             Action|Adventure|Sci-Fi|Thriller  
3                      Action|Adventure|Sci-Fi  
4                      Adventure|Comedy|Sci-Fi  
5  Adventure|Animation|Children|Comedy|Fantasy  
6             Action|Adventure|Sci-Fi|Th